https://app.diagrams.net/#G1kOLxSIqMuavf4peNEdkWelzKj6Rcp3nl

# Install requirements

In [9]:
import sys
import os

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install spotipy
!{sys.executable} -m pip install ipywidgets
!{sys.executable} -m pip install lyricsgenius
!{sys.executable} -m pip install NRCLex

isDatabricks = False

# Imports

In [10]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import StandardScaler

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import multiprocessing
from multiprocessing import Pool
from threading import Thread
import pandas as pd
import numpy as np
import requests
from urllib import request as urllib2
import json
from bs4 import BeautifulSoup
import re
import string


import lyricsgenius
from nrclex import NRCLex
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from sklearn.metrics.pairwise import cosine_similarity

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import uuid
import time

[nltk_data] Downloading package punkt to /home/valerio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/valerio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/valerio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Spotify connection

In [11]:
os.environ["SPOTIPY_CLIENT_ID"] = "9434b51d2fde473384c68c79130534f1"
os.environ["SPOTIPY_CLIENT_SECRET"] = "1bde23439629452e94212f78a3eb3d59"
os.environ["SPOTIPY_REDIRECT_URI"] = "https://developer.spotify.com/dashboard/applications/d65504514c0e40ef9c74cbfa0d1be767"
os.environ["SPOTIPY_CLIENT_USERNAME"] = "11154156130"
os.environ['PYSPARK_PYTHON']=sys.executable

In [12]:

client_id = "d65504514c0e40ef9c74cbfa0d1be767"
client_secret = "64ff22636db44b68992ef49b1c9eaaf4"
username = "11154156130"

scope = "playlist-modify-public playlist-modify-private playlist-read-private playlist-read-collaborative"
redirect_uri = "https://developer.spotify.com/dashboard/applications/9434b51d2fde473384c68c79130534f1"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
print(token)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

BQASUQn4__vvC1pKdalKxtJ2A3m34NDyBp59GU-SFcajyPXFz64hqiHmeGg5ESvwwK5f41jvNskt5k4riDZYfCtsShI_K63uy2T-SfmSztZ52sUycfaGagI0GvB_Heoznv6v6JP6-tqJIuqkRILY3O3x7L0R7JHZX-vKZGjJLOkSswvv7vzufwn54x5h2SNlNgBpRmtEcRrXP8vnw_iwturDCDXoJw


# Spark initialization

In [13]:
spark = SparkSession.builder.appName("BigData Project").getOrCreate()
sqlcontext = SQLContext(spark)

In [14]:
spark

# Grab spotify playlists and collect tracks information

### Functions and variables

In [15]:
ids = []
tracks = []


def getTrackIDs(user, playlist_id):
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items'][:20]: #[:50]:
        track = item['track']
        if track is not None:
            ids.append(track['id'])
    return ids

def getSpotifyTrackIDs(playlist_id):
    return getTrackIDs('spotify', playlist_id)

def getTrackFeatures(id):
    time.sleep(3)
    meta = sp.track(id)
    features = sp.audio_features(id)
    
    trackid = id
    
    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

    track = [trackid, name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    
    return track

def runChunk(thread,toCalcIds):
    time.sleep(3)
    for i in range(len(toCalcIds)):
        print("Thread: "+str(thread)+" processing Track Number: "+str(i)+" total: "+str(len(toCalcIds)))
        try: 
            track = getTrackFeatures(toCalcIds[i])
            tracks.append(track)
        except:
            pass
        
def storeDataframeOnFile(filename, df):
    if isDatabricks :
        %fs rm -r '/FileStore/tables/'+filename+'.csv'
        dbutils.fs.rm("/FileStore/tables/"+filename+".csv")
        df.write.format('com.databricks.spark.csv').option('header', 'true').save('dbfs:/FileStore/tables/'+filename+'.csv')
    else:
        df.write.format("com.databricks.spark.csv").option('header', 'true').save(filename+'.csv')
        
def loadDataframeFromFile(filename):
    if isDatabricks :
        df = sqlcontext.read.load('dbfs:/FileStore/tables/'+filename+'.csv', format='csv', header=True, inferSchema=True)
    else:
        df = sqlcontext.read.load(filename+'.csv', format='csv', header=True, inferSchema=True)
    return df

### Logic

In [ ]:
# Load spotify playlists

playlists = sp.user_playlists('spotify')
print(len(playlists))

playlist_ids = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        playlist_ids.append(playlist['id'])
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

In [ ]:
# get tracks from spotify playlists

for i in playlist_ids:
    ids = getTrackIDs('spotify', i) #playlist

In [ ]:
# number of tracks

print(len(ids))

In [ ]:
# loop over track ids to get audio features for each track

toCalcIds = ids[:5000]

runChunk(0,toCalcIds)

#chunks = np.array_split(toCalcIds, 10)
#print(len(chunks))

#try:
#    threads = list()
#    for index in range(10):
#        print("Main    : create and start thread "+str(index))
#        x = Thread(target=runChunk, args=(index,chunks[index]))
#        threads.append(x)
#        x.start()
#        
#    for index, thread in enumerate(threads):
#        print("Main    : before joining thread "+str(index))
#        thread.join()
#        print("Main    : thread done "+str(index))
#except:
#    print("Error: unable to start thread")
#
#while 1:
#    pass

In [ ]:
# create dataset
df = pd.DataFrame(tracks, columns = ['trackid', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])

In [ ]:
df = spark.createDataFrame(df)

In [ ]:
schema = df.printSchema()
schema

In [ ]:
storeDataframeOnFile('spotify', df)

In [ ]:
df = loadDataframeFromFile('spotify')

In [ ]:
# create a temporary view in order to use SQL statements
df.createOrReplaceTempView("songs")

In [ ]:
spark_dataset = sqlcontext.sql("SELECT * FROM songs")
spark_dataset = spark_dataset.dropna()

# MODEL 1 - K-Means clustering

In [ ]:
features = ['length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature']

vecAssembler = VectorAssembler(inputCols=features, outputCol="features")
new_df = vecAssembler.transform(spark_dataset)
new_df.show(2)

In [ ]:

scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(new_df)
data_scale_output=data_scale.transform(new_df)
data_scale_output.show(2)

In [ ]:

silhouette_score=[]

evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', metricName='silhouette', distanceMeasure='squaredEuclidean')

for i in range(2,20):
    
    KMeans_algo=KMeans(featuresCol='standardized', k=i)
    
    KMeans_fit=KMeans_algo.fit(data_scale_output)
    
    output=KMeans_fit.transform(data_scale_output)
    
    score=evaluator.evaluate(output)
    
    silhouette_score.append(score)
    
    print("Silhouette Score:",score)

In [ ]:
#Visualizing the silhouette scores in a plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize =(20,20))
ax.plot(range(2,20),silhouette_score)
ax.set_xlabel('k')
ax.set_ylabel('cost')

In [ ]:
mid_index = len(silhouette_score) // 2
mid_fit = silhouette_score[mid_index]
for i in range(mid_index-4, mid_index+4) :
    if mid_fit < silhouette_score[i]:
        print(i)
        print(mid_index+2)
        mid_fit = silhouette_score[i]
        mid_index = i

print(mid_fit)
print(mid_index)

In [ ]:
KMeans_algo=KMeans(featuresCol='standardized', k=mid_index+2)
KMeans_fit=KMeans_algo.fit(data_scale_output)
output=KMeans_fit.transform(data_scale_output)
centers = KMeans_fit.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

In [ ]:
summary = KMeans_fit.summary
print(summary.clusterSizes)

In [ ]:
output.toPandas()

In [ ]:
## Store results in spark

kmeansout = output.drop(output.features)
kmeansout.show(2)
kmeansout.createOrReplaceTempView("kmoutput")

In [ ]:
kmoutsql = sqlcontext.sql("SELECT * FROM kmoutput")
kmoutsql.toPandas()

# MODEL 2 - LYRICS EXTRACTION

In [ ]:
## https://genius.com/api-clients

lyrics_subjects = sqlcontext.sql("SELECT trackid, name, album, artist FROM songs")
lyrics_subjects.toPandas()

In [ ]:
os.environ['GENIUS_API_TOKEN']="ZqmoUelmG7iZBPQx8a1pn7QAWvThQsw2kmKRRivDle9auDcJg7fQYPy_RKx_jC6q"

In [ ]:
genius = lyricsgenius.Genius(os.environ.get("GENIUS_API_TOKEN"))
column_names = ["trackid", "fulltext"]
words_df = pd.DataFrame(columns = column_names)

#for row in df.rdd.collect():
#    count+=1
#    print("Track number "+str(count)+" of "+str(size))
    # Format a request URL for the Genius API
    #search_term = row.name+" "+row.artist

def getText(lyrics):
    for language in stopwords.fileids():
        stop_words=set(stopwords.words("english")) 
        text_not_stop = [ word for word in lyrics if word not in stop_words ]
    return text_not_stop

def retrieveLyrics(song):
    lyrics = song.lyrics
    lyrics = str(lyrics)
    lyrics = re.sub('\[.*\]','',lyrics) # Remove [Verse] and [Bridge] stuff
    lyrics = re.sub('\n{2}','',lyrics)  # Remove gaps between verses  
    lyrics = re.sub('\n',' ',lyrics) # Remove single \n
    lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
    lyrics = lyrics.lower()
    #lyrics = getText(lyrics) #to test
    return lyrics

def getWords(row):
    time.sleep(3)
    artist = genius.search_artist(row.artist, max_songs=1, sort="title")
    song = artist.song(row.name)
    if song is not None:
        #lyrics = song.lyrics
        #lyrics = str(lyrics)
        #lyrics = re.sub('\[.*\]','',lyrics) # Remove [Verse] and [Bridge] stuff
        #lyrics = re.sub('\n{2}','',lyrics)  # Remove gaps between verses  
        #lyrics = re.sub('\n',' ',lyrics) # Remove single \n
        #lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
        #lyrics = lyrics.lower()
        #word = lyrics.split()   #tokenizing all the lyrics body into individual words
        
        #for language in stopwords.fileids():
        #    stop_words=set(stopwords.words("english")) 
        #    text_not_stop = [ word for word in lyrics if word not in stop_words ]
        #print(retrieveLyrics(song))
        dat = pd.DataFrame.from_dict({'trackid': row.trackid, 'fulltext': [retrieveLyrics(song)] })
        return dat
    # words_df = words_df.append(words_df2, ignore_index = True)

#words_df = words_df.append(results)

In [ ]:
try:
    print(multiprocessing.cpu_count()-1)
    p = Pool(processes = (multiprocessing.cpu_count() - 1))
    words_df = words_df.append(p.map(getWords, df.rdd.collect()[:300]))
finally:
    p.close()
    p.join()

In [ ]:
words_df

In [ ]:
def getEmotions(words_df):
    column_names = ['trackid', 'words', 'Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']
    emolex_df = pd.DataFrame(columns = column_names)

    for index, row in words_df.iterrows():
        text = row['fulltext']
        text_object = NRCLex(text)

        if 'positive' in text_object.raw_emotion_scores.keys() :
            positive = text_object.raw_emotion_scores['positive']
        else:
            positive = 0

        if 'negative' in text_object.raw_emotion_scores.keys() :
            negative = text_object.raw_emotion_scores['negative']
        else:
            negative = 0

        if 'anger' in text_object.raw_emotion_scores.keys() :
            anger = text_object.raw_emotion_scores['anger']
        else:
            anger = 0

        if 'anticipation' in text_object.raw_emotion_scores.keys() :
            anticipation = text_object.raw_emotion_scores['anticipation']
        else:
            anticipation = 0

        if 'disgust' in text_object.raw_emotion_scores.keys() :
            disgust = text_object.raw_emotion_scores['disgust']
        else:
            disgust = 0

        if 'fear' in text_object.raw_emotion_scores.keys() :
            fear = text_object.raw_emotion_scores['fear']
        else:
            fear = 0

        if 'joy' in text_object.raw_emotion_scores.keys() :
            joy = text_object.raw_emotion_scores['joy']
        else:
            joy = 0

        if 'sadness' in text_object.raw_emotion_scores.keys() :
            sadness = text_object.raw_emotion_scores['sadness']
        else:
            sadness = 0

        if 'surprise' in text_object.raw_emotion_scores.keys() :
            surprise = text_object.raw_emotion_scores['surprise']
        else:
            surprise = 0

        if 'trust' in text_object.raw_emotion_scores.keys() :
            trust = text_object.raw_emotion_scores['trust']
        else:
            trust = 0

        emolex_df2 = {'trackid': row.trackid, 
                      'words': text_object.words,
                      'Positive': positive,
                      'Negative': negative,
                      'Anger': anger,
                      'Anticipation': anticipation,
                      'Disgust': disgust,
                      'Fear': fear,
                      'Joy': joy,
                      'Sadness': sadness,
                      'Surprise': surprise,
                      'Trust': trust }

        emolex_df = emolex_df.append(emolex_df2, ignore_index = True)
        
    return emolex_df


In [ ]:
emolex_df = getEmotions(words_df)

In [ ]:
scores_df=emolex_df[['Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']]
cosine_sim=cosine_similarity(scores_df,scores_df)
cosine_sim.shape

In [ ]:
cosine_sim[0]

In [ ]:
indices=pd.Series(emolex_df.index, emolex_df['trackid']).drop_duplicates()

In [ ]:
def get_recommendations(title,indices,cosine_sim):
    index=indices[title]
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    song_indices = [i[0] for i in sim_scores]
    return emolex_df['trackid'].iloc[song_indices]

In [ ]:
get_recommendations('4fzsfWzRhPawzqhX8Qt9F3',indices,cosine_sim)

# APPLICATION

In [ ]:
if isDatabricks:
    dbutils.widgets.removeAll()

In [ ]:
class PlayListAndName:
    def __init__(self, name, id):
        self.name = name
        self.id = id

In [ ]:
myuser = '11154156130'
playlists = sp.user_playlists(myuser) #vulgs3g7788xjy1yu1plzoart #11154156130
user_playlists_id = []
playlists_names = [] 
for item in playlists['items']:
    print(item['id'])
    user_playlists_id.append(PlayListAndName(item['name'], item['id']))
    playlists_names.append(item['name'])

In [ ]:
result_km = None

def calculateKM(name, create_playlist, emotions):
    
    ### BASIC RECCOMENDATION USING K-MEANS
    user_tracks_ids = []
    
    for plys in user_playlists_id:
        if plys.name == name:
            playlist = sp.user_playlist(myuser, plys.id)
            for item in playlist['tracks']['items']: #[:50]:
                track = item['track']
                if track is not None:
                    user_tracks_ids.append(track['id'])  
    user_tracks = []
    
    for i in range(len(user_tracks_ids)):
        sys.stdout.write("\rProcessing Track Number %i" % i)
        sys.stdout.write("\Tracks Number %i" % len(user_tracks_ids))
        sys.stdout.flush()
        try:
            track = getTrackFeatures(user_tracks_ids[i])
            user_tracks.append(track)
        except:
            pass
    print(user_tracks)   
    userdf = pd.DataFrame(user_tracks, columns = ['trackid', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
    userdf = spark.createDataFrame(userdf)
    userdf.createOrReplaceTempView(myuser+"songs")
    myuser_spark_dataset = sqlcontext.sql("SELECT * FROM "+myuser+"songs")
    myuser_spark_dataset = myuser_spark_dataset.dropna()
    new_myuser_df = vecAssembler.transform(myuser_spark_dataset)
    myuser_data_scale=scale.fit(new_myuser_df)
    myuser_data_scale_output=myuser_data_scale.transform(new_myuser_df)
    myuser_output=KMeans_fit.transform(myuser_data_scale_output)

    recommender_km = myuser_output.select(myuser_output["prediction"]).drop_duplicates()
    reccomender_result_km = kmoutsql.join(recommender_km, ["prediction"], "inner")
    result_km = reccomender_result_km.drop_duplicates().sort(reccomender_result_km.popularity.desc()).drop("standardized").limit(15)
    
    track_list = result_km.select('trackid').collect()
    
    ## USING LYRICS EMOTION
    if emotions :
        column_names = ["trackid", "fulltext"]
        user_words_df = pd.DataFrame(columns = column_names)

        try:
            p = Pool(processes = (multiprocessing.cpu_count() - 1))
            user_words_df = user_words_df.append(p.map(getWords, userdf.rdd.collect()))
        finally:
            p.close()
            p.join()
            
        user_words_df = getEmotions(user_words_df)
        user_scores_df=user_words_df[['Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']]
        user_cosine_sim=cosine_similarity(user_scores_df,scores_df)
        
        emotion_tracks = []
        for index, row in user_words_df.iterrows():
            user_indices=pd.Series(user_words_df.index, user_words_df['trackid']).drop_duplicates()
            em = get_recommendations(row['trackid'],user_indices,user_cosine_sim)
            print(em.values.tolist())
            emotion_tracks.extend(em.values.tolist())
    
        emotion_tracks_as_set = set(emotion_tracks)
        intersection = emotion_tracks_as_set.intersection(track_list)
        track_list = list(intersection)
    
    if len(track_list)==0 and emotions:
        track_list = emotion_tracks
        
    ## CREATE PLAYLIST SECTION
    if create_playlist :
        x = uuid.uuid1()
        playlist_name = "Generated playlist "+str(x)+"for user "+myuser
        playlistid = sp.user_playlist_create("11154156130", playlist_name, public=False, description='Generated playlist from ML Databricks')
        tracks_to_add = []
        for single_track in track_list :
            tracks_to_add.append(single_track)
        sp.user_playlist_add_tracks(myuser, playlistid['uri'], tracks_to_add)
        
    return track_list

In [ ]:
if isDatabricks :
    dbutils.widgets.dropdown("names", playlists_names[0], [playlists_names[x] for x in range(len(playlists_names))], "Choose a playlist for user "+myuser)
else:
    user_stats = interact_manual(calculateKM, name=widgets.Dropdown(options=playlists_names), create_playlist=widgets.Dropdown(options=[True, False]), emotions=widgets.Dropdown(options=[True, False]))

In [ ]:
try:
    val = getArgument("names")
except:
    pass
    
if isDatabricks:
    calculateKM(val)